In [97]:
from language_processing import *
import pprint as pp
import pandas as pd
import re
import nltk
from nltk import word_tokenize
from nltk import pos_tag
from dotenv import load_dotenv
import json
import os

In [64]:
load_dotenv()
nltk.download('punkt');
nltk.download('averaged_perceptron_tagger');
nltk.download('wordnet');
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /Users/vonbecker/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/vonbecker/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [65]:
# read unique exhibit phenomena into a list

phenomena = pd.read_csv('data/phenomena/phenomena.csv').squeeze('columns').to_list()
pp.pprint(phenomena)

['Natural Frequency',
 'Electric Fields and Forces',
 'Motion: Accelerated Gravitational Motion',
 'Afterimage',
 'Judgment',
 'Thin Films',
 'Frame of Reference',
 'Nodes and Antinodes',
 'Sublimation',
 'Bernoulli Effect',
 'Color Mixing: Subtractive',
 'Size-Distance',
 'Upwelling',
 'Nonlinear Behavior',
 'Information Processing and Encoding',
 'Resonance',
 'Perception: Spatial',
 'Attention',
 'Melting',
 'Visual Edge Effects',
 'Superposition',
 'Exponentials',
 'Surface Tension of Liquids',
 'Numbers',
 'Color: Complimentary',
 'Binocular Vision',
 'Crystallization',
 'Fatigue',
 'Color Vision',
 'Oscillation',
 'Iridescence',
 'Conservation of Angular Momentum',
 'Decomposition',
 'Polarization',
 'Evaporation',
 'Close Packing',
 'Ferromagnetism',
 'Parabolas',
 'Metacognition',
 'Albedo',
 'Depth Perception',
 'Waves: Transverse',
 'Probability',
 'Shadows',
 'Rhythm',
 'Motion: Composition of Perpendicular Motion',
 'Motion: Simple Harmonic Motion',
 'Social Loafing',
 'Gra

In [66]:
# read unique keywords into a list

keywords = pd.read_csv('data/keywords/keywords.csv').squeeze('columns').to_list()
pp.pprint(keywords)

['lenses',
 'center of gravity',
 'vibration',
 'weather',
 'identity',
 'cornsweet illusion',
 'acoustics',
 'evaporation',
 'chicken wire',
 'Ocean',
 'Sharks',
 'Neon',
 'Migrations',
 "Huygens' principle",
 'mirrors',
 'turbulence',
 'carbon dioxide',
 'parallel',
 'phase angle',
 'parabolas',
 'organisms',
 'cosmic rays',
 'simultaneous contrast',
 'series',
 'chaos',
 'amplification',
 'genetics',
 'lasers',
 'artwork',
 'membrane',
 "Newton's Laws",
 'reflections',
 'ratio',
 'circuit',
 'ping pong balls',
 'DNA',
 'levitation',
 'navigation',
 'curves',
 'illusions',
 'model organisms',
 'Whales',
 'gases',
 'portraits',
 'sun',
 'bicycle wheel',
 'speakers',
 'identical twins',
 'pendulums',
 'dissection',
 'eyes',
 'nerves',
 'wind',
 'zebrafish',
 'heat transfer',
 'volcanoes',
 'Marine',
 'Pacific',
 'complexity',
 'Turtles',
 'condensation',
 'appearance',
 'unicellular organisms',
 'tides',
 'ganzfeld effect',
 'afterimage',
 'fragmentation',
 'CMY retina',
 'balance',
 '

In [69]:
# clean up phenomena

# express 'color mixing: subtractive' as 'subtractive color mixing', etc.
# express 'motion: simple harmonic motion' as 'simple harmonic motion', etc.
# express 'motion: visual motion detection as 'visual motion detection'
# express 'this-that' and 'this/that' as 'this that'
# convert to lower case

phenomena_direct = []
for phenomenon in phenomena:
    this_phenom = phenomenon.lower()
    if ':' in phenomenon:
        split = re.split(': ', this_phenom)
        reordered = ' '.join(split[::-1])
        double_word = r'\b(\w+)\s+\1\b'
        separated_double = r'\b(\w+)\s(\w+)\s+\1\b'
        doubles = re.search(double_word, reordered)
        separated_doubles = re.search(separated_double, reordered)
        if doubles:
            reordered = reordered.replace(doubles.group(), doubles.group().split()[0])
        if separated_doubles:
            reordered = reordered.replace(separated_doubles.group(), ' '.join(separated_doubles.group().split()[:2]))
        this_phenom = reordered
    if any(x in phenomenon for x in ['-', '/']):
        split = re.split('-|/', this_phenom)
        this_phenom = ' '.join(split)
    phenomena_direct.append(this_phenom)

pp.pprint(phenomena_direct)

['natural frequency',
 'electric fields and forces',
 'accelerated gravitational motion',
 'afterimage',
 'judgment',
 'thin films',
 'frame of reference',
 'nodes and antinodes',
 'sublimation',
 'bernoulli effect',
 'subtractive color mixing',
 'size distance',
 'upwelling',
 'nonlinear behavior',
 'information processing and encoding',
 'resonance',
 'spatial perception',
 'attention',
 'melting',
 'visual edge effects',
 'superposition',
 'exponentials',
 'surface tension of liquids',
 'numbers',
 'complimentary color',
 'binocular vision',
 'crystallization',
 'fatigue',
 'color vision',
 'oscillation',
 'iridescence',
 'conservation of angular momentum',
 'decomposition',
 'polarization',
 'evaporation',
 'close packing',
 'ferromagnetism',
 'parabolas',
 'metacognition',
 'albedo',
 'depth perception',
 'transverse waves',
 'probability',
 'shadows',
 'rhythm',
 'composition of perpendicular motion',
 'simple harmonic motion',
 'social loafing',
 'gravitational forces',
 'torque

In [70]:
# clean up keywords
# express 'this-that' and 'this/that' as 'this that'
# convert to lower case

keywords_direct = []
for keyword in keywords:
    this_keyword = keyword.lower()
    if any(x in keyword for x in ['/']):
        split = re.split('-|/', this_keyword)
        this_keyword = ' '.join(split)
    keywords_direct.append(this_keyword)

pp.pprint(keywords_direct)

['lenses',
 'center of gravity',
 'vibration',
 'weather',
 'identity',
 'cornsweet illusion',
 'acoustics',
 'evaporation',
 'chicken wire',
 'ocean',
 'sharks',
 'neon',
 'migrations',
 "huygens' principle",
 'mirrors',
 'turbulence',
 'carbon dioxide',
 'parallel',
 'phase angle',
 'parabolas',
 'organisms',
 'cosmic rays',
 'simultaneous contrast',
 'series',
 'chaos',
 'amplification',
 'genetics',
 'lasers',
 'artwork',
 'membrane',
 "newton's laws",
 'reflections',
 'ratio',
 'circuit',
 'ping pong balls',
 'dna',
 'levitation',
 'navigation',
 'curves',
 'illusions',
 'model organisms',
 'whales',
 'gases',
 'portraits',
 'sun',
 'bicycle wheel',
 'speakers',
 'identical twins',
 'pendulums',
 'dissection',
 'eyes',
 'nerves',
 'wind',
 'zebrafish',
 'heat transfer',
 'volcanoes',
 'marine',
 'pacific',
 'complexity',
 'turtles',
 'condensation',
 'appearance',
 'unicellular organisms',
 'tides',
 'ganzfeld effect',
 'afterimage',
 'fragmentation',
 'cmy retina',
 'balance',
 '

In [71]:
# tokenize and pos tag each phenomenon

phenom_token_tag = []
for phenomenon in phenomena_direct:
    words = word_tokenize(phenomenon)
    tags = pos_tag(words)
    phenom_token_tag.append(tags)

pp.pprint(phenom_token_tag)

[[('natural', 'JJ'), ('frequency', 'NN')],
 [('electric', 'JJ'), ('fields', 'NNS'), ('and', 'CC'), ('forces', 'NNS')],
 [('accelerated', 'VBN'), ('gravitational', 'JJ'), ('motion', 'NN')],
 [('afterimage', 'NN')],
 [('judgment', 'NN')],
 [('thin', 'JJ'), ('films', 'NNS')],
 [('frame', 'NN'), ('of', 'IN'), ('reference', 'NN')],
 [('nodes', 'NNS'), ('and', 'CC'), ('antinodes', 'NNS')],
 [('sublimation', 'NN')],
 [('bernoulli', 'NNS'), ('effect', 'NN')],
 [('subtractive', 'JJ'), ('color', 'NN'), ('mixing', 'NN')],
 [('size', 'NN'), ('distance', 'NN')],
 [('upwelling', 'VBG')],
 [('nonlinear', 'JJ'), ('behavior', 'NN')],
 [('information', 'NN'),
  ('processing', 'NN'),
  ('and', 'CC'),
  ('encoding', 'NN')],
 [('resonance', 'NN')],
 [('spatial', 'JJ'), ('perception', 'NN')],
 [('attention', 'NN')],
 [('melting', 'VBG')],
 [('visual', 'JJ'), ('edge', 'NN'), ('effects', 'NNS')],
 [('superposition', 'NN')],
 [('exponentials', 'NNS')],
 [('surface', 'NN'), ('tension', 'NN'), ('of', 'IN'), ('li

In [76]:
# tokenize and tag each keyword

keyword_token_tag = []
for keyword in keywords_direct:
    words = word_tokenize(keyword)
    tags = pos_tag(words)
    keyword_token_tag.append(tags)

pp.pprint(keyword_token_tag)

[[('lenses', 'NNS')],
 [('center', 'NN'), ('of', 'IN'), ('gravity', 'NN')],
 [('vibration', 'NN')],
 [('weather', 'NN')],
 [('identity', 'NN')],
 [('cornsweet', 'NN'), ('illusion', 'NN')],
 [('acoustics', 'NNS')],
 [('evaporation', 'NN')],
 [('chicken', 'NN'), ('wire', 'NN')],
 [('ocean', 'NN')],
 [('sharks', 'NNS')],
 [('neon', 'NN')],
 [('migrations', 'NNS')],
 [('huygens', 'NNS'), ("'", 'POS'), ('principle', 'NN')],
 [('mirrors', 'NNS')],
 [('turbulence', 'NN')],
 [('carbon', 'NN'), ('dioxide', 'NN')],
 [('parallel', 'NN')],
 [('phase', 'NN'), ('angle', 'NN')],
 [('parabolas', 'NNS')],
 [('organisms', 'NNS')],
 [('cosmic', 'JJ'), ('rays', 'NNS')],
 [('simultaneous', 'JJ'), ('contrast', 'NN')],
 [('series', 'NN')],
 [('chaos', 'NN')],
 [('amplification', 'NN')],
 [('genetics', 'NNS')],
 [('lasers', 'NNS')],
 [('artwork', 'NN')],
 [('membrane', 'NN')],
 [('newton', 'NN'), ("'s", 'POS'), ('laws', 'NNS')],
 [('reflections', 'NNS')],
 [('ratio', 'NN')],
 [('circuit', 'NN')],
 [('ping', '

In [77]:
# define master list of search terms:
# join keywords and phenomena
# keep unique elements

all_token_tag = phenom_token_tag + keyword_token_tag

unique_token_tag = []
[unique_token_tag.append(x) for x in all_token_tag if x not in unique_token_tag]

print('{} unique phenomena'.format(len(phenom_token_tag)))
print('{} unique keywords'.format(len(keyword_token_tag)))
print('{} unique terms in phenemena and keywords combined'.format(len(unique_token_tag)))

144 unique phenomena
114 unique keywords
252 unique terms in phenemena and keywords combined


In [78]:
# investigate distribution of tag sequences for the search terms
# a vast majority of the terms are noun phrases

all_tag_seq = []
for item in all_token_tag:
    tag_seq = ' '.join([subitem[1] for subitem in item])
    all_tag_seq.append(tag_seq)
all_tag_seq_pd = pd.Series(all_tag_seq)

print(all_tag_seq_pd.value_counts())

NN                           104
NNS                           38
JJ NN                         28
NN NN                         23
JJ NNS                        12
NN IN NN                       7
VBG                            6
NN NNS                         5
JJ NN NN                       4
VBN JJ NN                      3
NNS NN                         2
NNS CC NNS                     2
NN VBG                         2
VBG NNS                        2
NN IN JJ NN                    2
NN NNS IN NN                   1
NNS POS NN                     1
JJ                             1
NN POS NNS                     1
VBG NN NNS                     1
JJ VBZ NNS NN NN NN NN NN      1
NNS TO VB DT JJ JJ             1
JJ JJ                          1
RB VBG                         1
NN CC NN                       1
NN NN NN                       1
NN POS NN                      1
NN JJ NN                       1
JJ NNS CC NNS                  1
NN NN IN NNS                   1
JJ NN NNS 

In [79]:
# define related search terms (and the pos tags) for each main search terms

all_plus_related = []

for item in all_token_tag:
    text = ' '.join([word[0] for word in item])
    data = {
        'text': text,
        'primary': item
    }
    adj_nouns = get_adj_nouns(item)
    noun_nouns = get_noun_noun(item)
    vb_nouns = get_vb_nouns(item)
    adj_verbs = get_adj_verbs(item)
    nouns = get_nouns(item)
    verbs = get_verbs(item)
    adjectives = get_adjectives(item)
    related_items = nouns
    related_items += verbs
    related_items += adjectives
    related_items += adj_nouns
    related_items += noun_nouns
    related_items += vb_nouns
    related_items = [x for x in related_items if x]
    data['related'] = related_items
    all_plus_related.append(data)

pp.pprint(all_plus_related)

[{'primary': [('natural', 'JJ'), ('frequency', 'NN')],
  'related': [[('frequency', 'NN')], [('natural', 'JJ')]],
  'text': 'natural frequency'},
 {'primary': [('electric', 'JJ'),
              ('fields', 'NNS'),
              ('and', 'CC'),
              ('forces', 'NNS')],
  'related': [[('fields', 'NNS')],
              [('forces', 'NNS')],
              [('electric', 'JJ')],
              [('electric', 'JJ'), ('fields', 'NNS')],
              [('electric', 'JJ'), ('forces', 'NNS')]],
  'text': 'electric fields and forces'},
 {'primary': [('accelerated', 'VBN'),
              ('gravitational', 'JJ'),
              ('motion', 'NN')],
  'related': [[('motion', 'NN')],
              [('accelerated', 'VBN')],
              [('gravitational', 'JJ')],
              [('gravitational', 'JJ'), ('motion', 'NN')]],
  'text': 'accelerated gravitational motion'},
 {'primary': [('afterimage', 'NN')], 'related': [], 'text': 'afterimage'},
 {'primary': [('judgment', 'NN')], 'related': [], 'text': '

In [92]:
# separate lists of primary search terms and related search terms
# keep only related search terms which don't coincide with a primary search term

primary_items = []
related_items = []
related_items_unique = []

for item in all_plus_related:
    primary_items += [item['primary']]
    related_items += item['related']

[related_items_unique.append(x) for x in related_items if x not in related_items_unique + primary_items];

primary_items_pd = pd.Series(primary_items)
related_items_pd = pd.Series(related_items_unique)

In [94]:
print('There are {} unique main search terms'.format(len(primary_items)))
print('There are {} unique related search terms, none of which coincides with a main search term'.format((len(related_items_unique))))

There are 258 unique main search terms
There are 216 unique related search terms, none of which coincides with a main search term


In [96]:
# save all_plus_related to json file
# save primary_items and related_items_unique to csv files

path = 'data/search_terms'
if not os.path.exists(path):
                os.makedirs(path)

all_filepath = os.path.join(path, 'primary_and_related.json')
with open(all_filepath, "w") as f:
    f.write(json.dumps(all_plus_related, indent=2))

primary_filepath = os.path.join(path, 'primary.csv')
primary_items_pd.to_csv(primary_filepath, index=False)

related_filepath = os.path.join(path, 'related.csv')
related_items_pd.to_csv(related_filepath, index=False)